In [1]:
import tkinter as tk
from PIL import ImageTk, Image
from tkinter import filedialog
import numpy as np
import tensorflow as tf
import core.utils as utils
from tensorflow.python.saved_model import tag_constants
import cv2
import time

weights="./checkpoints/yolov4-416"

/opt/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
def load_img():
    
    global img, image_data,panel_image
        
    for img_display in frame.winfo_children():
        img_display.destroy()

    image_data = filedialog.askopenfilename(initialdir="/", title="Choose an image",
                                       filetypes=(("all files", "*.*"), ("png files", "*.png"),("jpg files", "*.jpg"),("jpeg files", "*.jpeg")))
    
    basewidth = 400 
#     # Processing image for Display
    img = Image.open(image_data)
    wpercent = (basewidth / float(img.size[0]))
    hsize = int((float(img.size[1]) * float(wpercent)))
    img = img.resize((basewidth, hsize), Image.ANTIALIAS)
    img = ImageTk.PhotoImage(img)
    
    file_name = image_data.split('/')
    # To display file name
    panel = tk.Label(frame, text= str(file_name[len(file_name)-1]).upper()).pack()
    # To display choosen image
    panel_image = tk.Label(frame, image=img).pack()
    

In [3]:
def load_weights():
    img_label.config(text="Loading weights...")
    time.sleep(1)
    global saved_model_loaded
    saved_model_loaded = tf.saved_model.load(weights, tags=[tag_constants.SERVING])
    img_label.config(text="Weights Loaded...")


In [4]:
def classify():
    
    global image_data

    framework="tf"
    size=416
    tiny=False
    model="yolov4"
    output="./detections/"
    iou=0.45
    score=0.25
    dont_show=False
    
    img_label.config(text="Detection.....")

    print("Detection.....")
    input_size = size

    
    # Read Image
    original_image = cv2.imread(image_data)
    # Change color channel to RGB
    original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
    # Change Image size
    image_data = cv2.resize(original_image, (input_size, input_size))
    # devide image pixel /255 to keep it in between 0 and 1
    image_data = image_data / 255.

    images_data = []
    for i in range(1):
        images_data.append(image_data)
    images_data = np.asarray(images_data).astype(np.float32)


    infer = saved_model_loaded.signatures['serving_default']
    
    batch_data = tf.constant(images_data)
    pred_bbox = infer(batch_data)
    
    for key, value in pred_bbox.items():
        boxes = value[:, :, 0:4]
        pred_conf = value[:, :, 4:]

    boxes, scores, classes, valid_detections = tf.image.combined_non_max_suppression(
        boxes=tf.reshape(boxes, (tf.shape(boxes)[0], -1, 1, 4)),
        scores=tf.reshape(
            pred_conf, (tf.shape(pred_conf)[0], -1, tf.shape(pred_conf)[-1])),
        max_output_size_per_class=50,
        max_total_size=50,
        iou_threshold=iou,
        score_threshold=score
    )
    
    pred_bbox = [boxes.numpy(), scores.numpy(), classes.numpy(), valid_detections.numpy()]
    
    # Get prediction text and predicted image
    img_text, image, class_names= utils.draw_bbox(original_image, pred_bbox)
    
    image = Image.fromarray(image.astype(np.uint8))
    if not dont_show:
        image.show()
    image = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2RGB)
    
    img_label.config(text="".join(class_names))
    print("".join(class_names))
    cv2.imwrite(output + 'detection.png', image)  
   

In [5]:
root = tk.Tk()
root.title('Fruits & Vegetable Detector')
# root.iconbitmap('class.ico')
root.configure(background='lightblue')
root.resizable(True, True)
tit = tk.Label(root, text="Fruits & Vegetable Detector",padx=25, pady=6, font=("", 12)).pack()
canvas = tk.Canvas(root, height=500, width=500, bg='lightgreen')
canvas.pack()
frame = tk.Frame(root, bg='white')
frame.place(relwidth=0.8, relheight=0.8, relx=0.1, rely=0.1)

global img_label

# Load weights button
load_weights = tk.Button(root, text='Load Weights',
                        padx=35, pady=10,
                        fg="green", bg="blue", command=load_weights)
load_weights.pack(side=tk.TOP)

# chose image button
chose_image = tk.Button(root, text='Choose Image',
                        padx=35, pady=10,
                        fg="green", bg="blue", command=load_img)
chose_image.pack(side=tk.LEFT)

# Classify Image button
class_image = tk.Button(root, text='Classify Image',
                        padx=35, pady=10,
                        fg="green", bg="blue", command=classify)
class_image.pack(side=tk.RIGHT)

img_label = tk.Label(root, text="Select Image", padx=25, font=("", 12), fg="green")
img_label.pack(pady=6)


root.mainloop()

Detection.....
apple: 0.93
apple: 0.93
Detection.....
apple-bag: 0.96
tomato-bag: 0.46
apple-bag: 0.96tomato-bag: 0.46
Detection.....
banana-bag: 1.00
banana-bag: 1.00
Detection.....
blackberries: 1.00
blackberries: 1.00
Detection.....
chilli: 1.00
chilli: 1.00
Detection.....
grapes-bag: 0.99
grapes-bag: 0.99
Detection.....
grapes-bag: 0.96
grapes-bag: 0.96
Detection.....
lemon-bag: 0.80
chilli-bag: 0.59
lemon-bag: 0.80chilli-bag: 0.59
Detection.....
raspberry: 0.72
raspberry: 0.72
Detection.....
tomato: 0.74
tomato: 0.74
Detection.....
tomato-bag: 1.00
tomato-bag: 1.00
